In [1]:
import pandas as pd
# Set max columns to display
pd.set_option('display.max_columns', None)

import numpy as np
from sklearn import preprocessing as pre
import matplotlib.pyplot as plt
from scipy import stats
import statsmodels.api as sm
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score



C:\Users\austinsh\AppData\Local\Temp\ipykernel_3656\1304120184.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:

# # Importing CSV files
# df_CDunit = pd.read_csv('https://raw.githubusercontent.com/saust1/Project-OptiC4/main/1%20Preprocess/Continuous%20Data/cont_554Data_clean.csv')
# df_AlCon = pd.read_csv('https://raw.githubusercontent.com/saust1/Project-OptiC4/main/1%20Preprocess/Continuous%20Data/cont_425Data_clean.csv')
# df_FB554 = pd.read_csv('https://raw.githubusercontent.com/saust1/Project-OptiC4/main/1%20Preprocess/Continuous%20Data/cont_unitData_clean.csv')


In [3]:
# # Importing CSV files
# bordeCode directory
df_CDunit = pd.read_csv(r'C:\Users\austinsh\Project-OptiC4\IV Optimize\Testing Data\Continuous Data\cont_unitData_clean.csv')
df_AlCon = pd.read_csv(r'C:\Users\austinsh\Project-OptiC4\IV Optimize\Testing Data\Continuous Data\cont_425Data_clean.csv')
df_FB554 = pd.read_csv(r'C:\Users\austinsh\Project-OptiC4\IV Optimize\Testing Data\Continuous Data\cont_554Data_clean.csv')



In [4]:
print(df_CDunit.describe())
print(df_AlCon.describe())
print(df_FB554.describe())


            DI55102       DI55152       DI55580       FC55003       FC55009  \
count  47714.000000  47714.000000  47714.000000  47714.000000  47714.000000   
mean       0.943832      0.932738      0.999463   6004.739389    840.214589   
std        0.054113      0.030251      0.050624    845.117946    612.586244   
min        0.800002      0.836900      0.848476   2868.570000      0.000000   
25%        0.909931      0.912513      0.965970   5491.450000    289.946250   
50%        0.950101      0.932759      0.998735   6021.665000    825.158000   
75%        0.984713      0.952295      1.034800   6565.487500   1318.492500   
max        1.061510      1.025180      1.144600   9134.330000   2674.790000   

            FC55552       FC55569       FC55576      FFC55553      FFC55555  \
count  47714.000000  47714.000000  47714.000000  47714.000000  47714.000000   
mean   35995.032821   6606.136206    368.910092      1.000340      0.771854   
std     4921.171720    384.746947    256.284600    

In [5]:
print("Data type for 'Date' column in df_CDunit:", df_CDunit['Date'].dtypes)
print("Data type for 'Date' column in df_FB554:", df_FB554['Date'].dtypes)
print("Data type for 'Date' column in df_AlCon:", df_AlCon['Date'].dtypes)


Data type for 'Date' column in df_CDunit: object
Data type for 'Date' column in df_FB554: object
Data type for 'Date' column in df_AlCon: object


In [6]:
df_CDunit['Date'] = pd.to_datetime(df_CDunit['Date'], errors='coerce')
df_FB554['Date'] = pd.to_datetime(df_FB554['Date'], errors='coerce')
df_AlCon['Date'] = pd.to_datetime(df_AlCon['Date'], errors='coerce')


In [7]:
print("Data type for 'Date' column in df_CDunit:", df_CDunit['Date'].dtypes)
print("Data type for 'Date' column in df_FB554:", df_FB554['Date'].dtypes)
print("Data type for 'Date' column in df_AlCon:", df_AlCon['Date'].dtypes)


Data type for 'Date' column in df_CDunit: datetime64[ns]
Data type for 'Date' column in df_FB554: datetime64[ns]
Data type for 'Date' column in df_AlCon: datetime64[ns]


In [8]:
print(df_CDunit.columns)
print(df_FB554.columns)
print(df_AlCon.columns)


Index(['Date', 'DI55102', 'DI55152', 'DI55580', 'FC55003', 'FC55009',
       'FC55552', 'FC55569', 'FC55576', 'FFC55553', 'FFC55555', 'LC55557',
       'LC90366', 'LC90368', 'PI55020', 'TC55552', 'TI55021'],
      dtype='object')
Index(['Date', 'Butanol', 'Decanol'], dtype='object')
Index(['Date', '425_pct_Al', 'Al2O3', 'M_Value', 'C4_pct_Eth', 'C4_pct_H2O',
       'HydWtr_pct_Ammonia', 'C4_pct_Hex', 'HydWtr_Na2O'],
      dtype='object')


In [9]:
def apply_rolling_average_to_df(df, rolling_size):
    # Ensure 'Date' is the index if it's not already
    if df.index.name != 'Date':
        df = df.set_index('Date')

    # Apply rolling average to all columns
    rolled_df = df.rolling(window=rolling_size, min_periods=1).mean()

    # Reset index to make 'Date' a column again
    rolled_df = rolled_df.reset_index()

    return rolled_df



In [10]:
def apply_time_shift_by_hours(df, shift_hours):
    """
    Shifts the DataFrame's datetime index by the specified number of hours.

    :param df: DataFrame with 'Date' as its datetime index or column.
    :param shift_hours: Number of hours to shift. Can be positive (forward) or negative (backward).
    :return: Shifted DataFrame.
    """
    # Convert 'Date' to datetime and set as index if it's not already
    if df.index.name != 'Date':
        df['Date'] = pd.to_datetime(df['Date'], errors='coerce')
        df = df.set_index('Date')

    # Ensure the index is a DatetimeIndex
    df.index = pd.to_datetime(df.index)

    # Shift the DataFrame's index by the specified number of hours
    df.index = df.index + pd.Timedelta(hours=shift_hours)

    return df

In [11]:
# # Usage Examples
# shift_hours_AlCon = 1  # Negative shift for df_AlCon (e.g., -5 hours backward)
# shift_hours_FB554 = 5   # Positive shift for df_FB554 (e.g., 5 hours forward)

# shifted_df_AlCon = apply_time_shift_by_hours(df_AlCon, shift_hours_AlCon)
# print("Shifted df_AlCon:")
# print(shifted_df_AlCon.head())

# shifted_df_FB554 = apply_time_shift_by_hours(df_FB554, shift_hours_FB554)
# print("\nShifted df_FB554:")
# print(shifted_df_FB554.head())

In [12]:
def join_df_FB554_to_df_CDunit(df_CDunit, df_FB554):
    # Reset index if 'Date' is the index
    if df_CDunit.index.name == 'Date':
        df_CDunit = df_CDunit.reset_index()
    if df_FB554.index.name == 'Date':
        df_FB554 = df_FB554.reset_index()

    # Ensure 'Date' columns are datetime objects and sort DataFrames
    df_CDunit['Date'] = pd.to_datetime(df_CDunit['Date'], errors='coerce')
    df_FB554['Date'] = pd.to_datetime(df_FB554['Date'], errors='coerce')

    df_CDunit = df_CDunit.dropna(subset=['Date']).sort_values('Date')
    df_FB554 = df_FB554.dropna(subset=['Date']).sort_values('Date')

    # Perform merge_asof
    combined_df = pd.merge_asof(df_FB554, df_CDunit, on='Date', direction='nearest')

    return combined_df

def join_df_AlCon_to_combined_df(combined_df, df_AlCon):
    # Reset index if 'Date' is the index
    if combined_df.index.name == 'Date':
        combined_df = combined_df.reset_index()
    if df_AlCon.index.name == 'Date':
        df_AlCon = df_AlCon.reset_index()

    # Ensure 'Date' columns are datetime objects and sort DataFrames
    combined_df['Date'] = pd.to_datetime(combined_df['Date'], errors='coerce')
    df_AlCon['Date'] = pd.to_datetime(df_AlCon['Date'], errors='coerce')

    combined_df = combined_df.dropna(subset=['Date']).sort_values('Date')
    df_AlCon = df_AlCon.dropna(subset=['Date']).sort_values('Date')

    # Perform merge_asof
    combined_df_all = pd.merge_asof(df_AlCon, combined_df, on='Date', direction='nearest')
    
    return combined_df_all


In [13]:
print("Data type for 'Date' column in df_CDunit:", df_CDunit['Date'].dtypes)
print("Data type for 'Date' column in df_FB554:", df_FB554['Date'].dtypes)
print("Data type for 'Date' column in df_AlCon:", df_AlCon['Date'].dtypes)



Data type for 'Date' column in df_CDunit: datetime64[ns]
Data type for 'Date' column in df_FB554: datetime64[ns]
Data type for 'Date' column in df_AlCon: datetime64[ns]


In [14]:
def process_data_limited():
    # Apply the specific rolling average directly
    rolled_df_CDunit = apply_rolling_average_to_df(df_CDunit, 8)
    rolled_df_FB554 = apply_rolling_average_to_df(df_FB554, 4)
    rolled_df_AlCon = apply_rolling_average_to_df(df_AlCon, 2)

    # Apply the specific time shifts directly
    rolled_df_AlCon_shifted = apply_time_shift_by_hours(rolled_df_AlCon, -1) # Assuming apply_time_shift_by_hours handles negative shifts correctly
    rolled_df_FB554_shifted = apply_time_shift_by_hours(rolled_df_FB554, 1)

    # Combine df_CDunit and df_FB554 to create combined_df
    combined_df = join_df_FB554_to_df_CDunit(rolled_df_CDunit, rolled_df_FB554_shifted)

    # Combine combined_df with rolled_df_AlCon to create combined_df_all
    combined_df_all = join_df_AlCon_to_combined_df(combined_df, rolled_df_AlCon_shifted)

    # At this point, combined_df_all is the DataFrame with the data processed by the specified shifts and averages
    return combined_df_all


In [15]:
# Make sure all your helper functions and initial DataFrames (df_CDunit, df_FB554, df_AlCon) are correctly defined

# Now, call the modified process_data function to get the processed DataFrame
final_dataset = process_data_limited()

# Inspect the final_dataset
print(final_dataset.head())  # Print the first few rows to inspect the dataset


                 Date  425_pct_Al    Al2O3  M_Value  C4_pct_Eth  C4_pct_H2O  \
0 2012-05-16 16:00:00    6.307630  11.3300  3.50893    2.508420    21.96340   
1 2012-05-16 18:00:00    6.308335  11.3750  3.50863    2.511135    21.96865   
2 2012-05-16 19:00:00    6.309390  11.4425  3.50818    2.515210    21.97655   
3 2012-05-16 20:00:00    6.310090  11.4875  3.50788    2.517925    21.98185   
4 2012-05-16 21:00:00    6.310790  11.5325  3.50758    2.520640    21.98715   

   HydWtr_pct_Ammonia  C4_pct_Hex  HydWtr_Na2O    Butanol   Decanol   DI55102  \
0            0.908765    0.659256     2.712180  54.858300  0.722511  0.970496   
1            0.907643    0.659930     2.805415  54.858300  0.722511  0.970496   
2            0.905959    0.660942     2.945270  54.858300  0.722511  0.970496   
3            0.904837    0.661616     3.038505  51.190050  0.733904  0.970653   
4            0.903715    0.662291     3.131740  48.744567  0.741499  0.970307   

    DI55152   DI55580  FC55003      FC

In [16]:
# Inspect the final_dataset
print(final_dataset.describe())  # Print the first few rows to inspect the dataset

                                Date    425_pct_Al         Al2O3  \
count                          47714  47714.000000  47714.000000   
mean   2016-07-09 15:30:57.643458816      6.148541     11.355834   
min              2012-05-16 16:00:00      5.174085      9.531050   
25%              2014-08-02 21:15:00      6.006720     11.084600   
50%              2016-07-31 07:30:00      6.142755     11.483750   
75%              2018-05-27 12:45:00      6.290847     11.701963   
max              2022-03-24 11:00:00      7.134450     12.879900   
std                              NaN      0.255252      0.570302   

            M_Value    C4_pct_Eth    C4_pct_H2O  HydWtr_pct_Ammonia  \
count  47714.000000  47714.000000  47714.000000        47714.000000   
mean       3.592181      1.210588     21.161871            0.970035   
min        1.347070      0.064828      8.149630            0.374388   
25%        3.500671      0.627316     18.776275            0.876725   
50%        3.587543      1.10755

In [17]:
import pandas as pd

# Assuming the 'Date' column is a string, you can extract the year using the str accessor
# final_dataset['Year'] = final_dataset['Date'].str[:4]

# Alternatively, if the 'Date' column is already a datetime object, you can extract the year directly
final_dataset['Year'] = final_dataset['Date'].dt.year

# Now count the records for each year
record_counts_by_year = final_dataset['Year'].value_counts().sort_index()

# Output the results
print(record_counts_by_year)


Year
2012    2151
2013    6740
2014    5852
2015    5972
2016    6251
2017    6046
2018    6281
2019    4871
2020    3188
2022     362
Name: count, dtype: int64


In [19]:
# Save DataFrame to CSV file in the same directory as the Jupyter Notebook
# model_results.to_csv('merged_data'.csv', index=False)
                     
# Save DataFrame to CSV file in the same directory as the Jupyter Notebook
# df_CD.to_csv(r'C:\Users\steve\OneDrive\1. BAIUTEK\Project-OptiC4\1 Preprocess\Continuous Data\contData_all.csv', index=False)            

final_dataset.to_csv(r'C:\Users\austinsh\Project-OptiC4\IV Optimize\Testing Data\Merge Data\merged_data.csv', index=False)


In [20]:
from datetime import datetime

# Get the current date and time
current_date_time = datetime.now()

# Print the current date and time
print(current_date_time)


2024-05-07 00:04:45.385081
